In [1]:
from pyspark.sql import SparkSession

In [3]:
MAX_MEMORY="5g"
spark = SparkSession.builder.appName("taxi-fare-prediciton")\
                .config("spark.executor.memory", MAX_MEMORY)\
                .config("spark.driver.memory", MAX_MEMORY)\
                .getOrCreate()

In [4]:
trip_files = "/Users/keon/fastcampus/data-engineering/01-spark/data/trips/*"

In [5]:
trips_df = spark.read.csv(f"file:///{trip_files}", inferSchema=True, header=True)

In [6]:
trips_df.printSchema()

root
 |-- VendorID: integer (nullable = true)
 |-- tpep_pickup_datetime: string (nullable = true)
 |-- tpep_dropoff_datetime: string (nullable = true)
 |-- passenger_count: integer (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- RatecodeID: integer (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- payment_type: integer (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- congestion_surcharge: double (nullable = true)



In [7]:
trips_df.createOrReplaceTempView("trips")

In [8]:
query = """
SELECT 
    trip_distance,
    total_amount
FROM
    trips
WHERE
    total_amount < 5000
    AND total_amount > 0
    AND trip_distance > 0
    AND trip_distance < 500
    AND passenger_count < 4
    AND TO_DATE(tpep_pickup_datetime) >= '2021-01-01'
    AND TO_DATE(tpep_pickup_datetime) < '2021-08-01'
"""

In [9]:
data_df = spark.sql(query)
data_df.createOrReplaceTempView("data")

In [10]:
data_df.show()

+-------------+------------+
|trip_distance|total_amount|
+-------------+------------+
|          2.1|        11.8|
|          0.2|         4.3|
|         14.7|       51.95|
|         10.6|       36.35|
|         4.94|       24.36|
|          1.6|       14.15|
|          4.1|        17.3|
|          5.7|        21.8|
|          9.1|        28.8|
|          2.7|       18.95|
|         6.11|        24.3|
|         1.21|       10.79|
|          7.4|       33.92|
|         1.01|        10.3|
|         0.73|       12.09|
|         1.17|       12.36|
|         0.78|        9.96|
|         1.66|        12.3|
|         0.93|         9.3|
|         1.16|       11.84|
+-------------+------------+
only showing top 20 rows



In [12]:
data_df.describe().show()

+-------+------------------+------------------+
|summary|     trip_distance|      total_amount|
+-------+------------------+------------------+
|  count|          13126040|          13126040|
|   mean|2.8820930920521075|17.973158757869456|
| stddev|3.8203064806711784|12.975904680786615|
|    min|              0.01|              0.01|
|    max|             475.5|            4973.3|
+-------+------------------+------------------+



In [13]:
train_df, test_df = data_df.randomSplit([0.8, 0.2], seed=1)

In [14]:
print(train_df.count())
print(test_df.count())

10500134
2625906


In [15]:
from pyspark.ml.feature import VectorAssembler

In [17]:
vassembler = VectorAssembler(inputCols=["trip_distance"], outputCol="features")

In [18]:
vtrain_df = vassembler.transform(train_df)

In [19]:
vtrain_df.show()

+-------------+------------+--------+
|trip_distance|total_amount|features|
+-------------+------------+--------+
|         0.01|         3.3|  [0.01]|
|         0.01|         3.3|  [0.01]|
|         0.01|         3.3|  [0.01]|
|         0.01|         3.3|  [0.01]|
|         0.01|         3.3|  [0.01]|
|         0.01|         3.3|  [0.01]|
|         0.01|         3.3|  [0.01]|
|         0.01|         3.3|  [0.01]|
|         0.01|         3.3|  [0.01]|
|         0.01|         3.3|  [0.01]|
|         0.01|         3.3|  [0.01]|
|         0.01|         3.3|  [0.01]|
|         0.01|         3.3|  [0.01]|
|         0.01|         3.3|  [0.01]|
|         0.01|         3.3|  [0.01]|
|         0.01|         3.3|  [0.01]|
|         0.01|         3.3|  [0.01]|
|         0.01|         3.3|  [0.01]|
|         0.01|         3.3|  [0.01]|
|         0.01|         3.3|  [0.01]|
+-------------+------------+--------+
only showing top 20 rows



In [20]:
from pyspark.ml.regression import LinearRegression

In [22]:
lr = LinearRegression(
    maxIter=50,
    labelCol="total_amount",
    featuresCol="features"
)

In [23]:
model = lr.fit(vtrain_df)

In [24]:
vtest_df = vassembler.transform(test_df)

In [25]:
prediction = model.transform(vtest_df)

In [26]:
prediction.show()

+-------------+------------+--------+----------------+
|trip_distance|total_amount|features|      prediction|
+-------------+------------+--------+----------------+
|         0.01|         3.3|  [0.01]|9.41325139745787|
|         0.01|         3.3|  [0.01]|9.41325139745787|
|         0.01|         3.3|  [0.01]|9.41325139745787|
|         0.01|         3.3|  [0.01]|9.41325139745787|
|         0.01|         3.3|  [0.01]|9.41325139745787|
|         0.01|         3.3|  [0.01]|9.41325139745787|
|         0.01|         3.3|  [0.01]|9.41325139745787|
|         0.01|         3.3|  [0.01]|9.41325139745787|
|         0.01|         3.3|  [0.01]|9.41325139745787|
|         0.01|         3.3|  [0.01]|9.41325139745787|
|         0.01|         3.3|  [0.01]|9.41325139745787|
|         0.01|         3.3|  [0.01]|9.41325139745787|
|         0.01|         3.3|  [0.01]|9.41325139745787|
|         0.01|         3.3|  [0.01]|9.41325139745787|
|         0.01|         3.8|  [0.01]|9.41325139745787|
|         

In [28]:
model.summary.rootMeanSquaredError

6.264517742357578

In [29]:
model.summary.r2

0.7677192474949946

In [34]:
from pyspark.sql.types import DoubleType
distance_list = [1.1]
distance_df = spark.createDataFrame(distance_list, DoubleType()).toDF("trip_distance")

In [35]:
distance_df.show()

+-------------+
|trip_distance|
+-------------+
|          1.1|
|          5.5|
|         10.5|
|         30.0|
+-------------+



In [36]:
vdistance_df = vassembler.transform(distance_df)

In [37]:
vdistance_df.show()

+-------------+--------+
|trip_distance|features|
+-------------+--------+
|          1.1|   [1.1]|
|          5.5|   [5.5]|
|         10.5|  [10.5]|
|         30.0|  [30.0]|
+-------------+--------+



In [39]:
model.transform(vdistance_df).show()

+-------------+--------+------------------+
|trip_distance|features|        prediction|
+-------------+--------+------------------+
|          1.1|   [1.1]|12.661823820190449|
|          5.5|   [5.5]|25.775327178010023|
|         10.5|  [10.5]|40.677035539168635|
|         30.0|  [30.0]| 98.79369814768721|
+-------------+--------+------------------+

